# Import Data

In [132]:
import numpy as np
from netCDF4 import Dataset
directory="/Users/lambda/Documents/Code/CaliforniaDrought2016/Data/"
#file=directory+"Pressure/hgt.mon.mean.nc"
#nc=Dataset(file,'r')
#pressure=nc.variables["hgt"][0:816,7,0:48,48:112]
file=directory+"Pressure/slp.mon.mean.nc"
nc=Dataset(file,'r')
#pressure=nc.variables["slp"][0:815,0:48,48:112]
pressure=nc.variables["slp"][0:816,0:48,48:112]
lat=nc.variables["lat"][:]
lon=nc.variables["lon"][:]
from numpy import loadtxt
CAMonthTotal= loadtxt(directory+'Precipitation/CAMonthTotal.txt')
ORMonthTotal= loadtxt(directory+'Precipitation/ORMonthTotal.txt')
WCMonthTotal= loadtxt(directory+'Precipitation/WCMonthTotal.txt')
CAMonthTotal=CAMonthTotal.reshape((816, 1))
ORMonthTotal=ORMonthTotal.reshape((816, 1))
WCMonthTotal=WCMonthTotal.reshape((816, 1))
CAMonthTotal=CAMonthTotal[0:816,:]
rlat=48
rlon=64

In [133]:
y_data=CAMonthTotal
x=pressure[0:3,:,:]
y=y_data[0:3,:]
for i in range(len(CAMonthTotal)/12):
    x=np.append(x,pressure[10+12*i:15+12*i,:,:],axis=0)
    y=np.append(y,y_data[10+12*i:15+12*i,:],axis=0)
l=len(x_data)
x_data=x.flatten()
y_data=y
    
x_data=np.reshape((x_data-np.mean(x_data))/np.std(x_data),(l,rlat,rlon,1))
y_data=(y_data-np.mean(y_data))/np.std(y_data)

# Set Computing Graph

### HyperParameters

In [134]:
ptraining=224
ctraining=[]
ctest=[]
#Convolution Layer Parameters
wc1=3
nc1=2
wc2=5
nc2=1
wp1=2
wp2=2
fl=(rlat-wc1-wc2-wp1-wp2+4)*(rlon-wc1-wc2-wp1-wp2+4)*nc2
wfc=50
wfc2=10

### Graph

In [135]:
import tensorflow as tf
tf.reset_default_graph()
tf.set_random_seed(15)
x = tf.placeholder('float', [None, rlat,rlon,1])
y = tf.placeholder('float',[None,1])
def convolutional_neural_network(x):
    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='VALID')
    def maxpool2d(x,W):
        return tf.nn.max_pool(x, [1,W,W,1], strides=[1,1,1,1], padding='VALID')
    weights = {'W_conv1':tf.Variable(tf.random_normal([wc1,wc1,1,nc1],stddev=0.001,name="W_conv1")),
               'W_conv2':tf.Variable(tf.random_normal([wc2,wc2,nc1,nc2],stddev=0.001)),
               'W_fc':tf.Variable(tf.random_normal([fl,wfc],stddev=0.001)),
               'W_fc2':tf.Variable(tf.random_normal([wfc,wfc2],stddev=0.001)),
               'out':tf.Variable(tf.random_normal([wfc2, 1],stddev=0.001))}

    biases = { 'b_conv1':tf.Variable(tf.random_normal([nc1],stddev=0.001)),
               'b_conv2':tf.Variable(tf.random_normal([nc2],stddev=0.001)),
               'b_fc':tf.Variable(tf.random_normal([wfc],stddev=0.001)),
               'b_fc2':tf.Variable(tf.random_normal([wfc2],stddev=0.001)),
               'out':tf.Variable(tf.random_normal([1],stddev=0.001))}
    
    
    conv1 = conv2d(x, weights['W_conv1']) + biases['b_conv1']
    pool1 = maxpool2d(conv1,wp1)

    conv2 = conv2d(pool1, weights['W_conv2']) + biases['b_conv2']
    pool2 = maxpool2d(conv2,wp1)

    fc = tf.reshape(pool2,[-1, fl])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc2 = tf.nn.relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    output = tf.matmul(fc2, weights['out'])+biases['out'] 
    return output

In [136]:
prediction = convolutional_neural_network(x)
cost = tf.reduce_mean(tf.square(prediction-y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)
all_saver = tf.train.Saver()

# Train

In [137]:
def train_neural_network(x):
    batch=120
    epoches=1000
    gtestcost=np.inf
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for epoch in range(epoches):
            for b in range(ptraining/batch):
                epoch_x=x_data[b*batch:(b+1)*batch,:,:,:]
                epoch_y=y_data[b*batch:(b+1)*batch,:]
                sess.run(optimizer, feed_dict={x: epoch_x, y: epoch_y})
                trainingcost=sess.run(cost, feed_dict={x: epoch_x, y: epoch_y})
                ctraining.append(trainingcost)
            testcost=sess.run(cost, feed_dict={x: x_data[ptraining:,:,:,:], 
                                                   y: y_data[ptraining:,:]})
            print testcost
            if testcost < gtestcost and (testcost<0.25):
                gtestcost=testcost
                all_saver.save(sess, 'CNN_Trained')
            ctest.append(testcost)
        result=sess.run(prediction,feed_dict={x: x_data})
        sess.close()
    return result 

In [138]:
prediction_batch=train_neural_network(x)

1.03714
1.03721
1.03728
1.03736
1.03744
1.03752
1.0376
1.03768
1.03777
1.03786
1.03796
1.03805
1.03815
1.03824
1.03832
1.03838
1.03841
1.0384
1.03831
1.03812
1.0378
1.03728
1.03651
1.03541
1.03389
1.03184
1.02914
1.02569
1.0215
1.01659
1.01082
1.00428
0.996946
0.988811
0.979768
0.971001
0.961545
0.952192
0.944721
0.939777
0.931111
0.913031
0.89426
0.880156
0.864648
0.848004
0.823199
0.798073
0.777126
0.758344
0.740526
0.729701
0.719917
0.709429
0.703505
0.704014
0.709762
0.716337
0.725685
0.733217
0.73905
0.750256
0.763171
0.770959
0.782567
0.806498
0.832888
0.853696
0.888656
0.936963
0.963889
0.992068
1.03446
1.04823
1.05779
1.12586
1.13924
1.17547
1.22936
1.23481
1.25453
1.27768
1.235
1.24107
1.24548
1.24707
1.27968
1.28577
1.29217
1.30479
1.27032
1.36008
1.2294
1.38151
1.39485
1.30696
1.44453
1.43112
1.338
1.4427
1.44767
1.38158
1.4485
1.50334
1.43216
1.44361
1.52355
1.47608
1.44479
1.51083
1.52729
1.46603
1.51493
1.57246
1.49382
1.52423
1.6009
1.51387
1.51848
1.58923
1.55633
1.5259

KeyboardInterrupt: 

In [139]:
[np.min(ctest),np.min(ctraining)]

[0.70350492, 0.18107601]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(ctraining[0:800:4], label="training")
plt.plot(ctest,label="test")
plt.title("Training Cost")
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

# Result

In [ ]:
with tf.Session() as sess:
    # Initialize variables
    sess.run(tf.global_variables_initializer())
    # Restore model weights from previously saved model
    load_path =all_saver.restore(sess, '/Users/lambda/Documents/Code/Lib/TensorFlow/CNN_Trained')
    trainingresult=sess.run(prediction,feed_dict={x:x_data[0:ptraining,:,:,:]})
    testresult=sess.run(prediction,feed_dict={x:x_data[ptraining:,:,:,:]})

In [ ]:
import matplotlib.pyplot as plt
plt.plot(trainingresult,y_data[0:ptraining,:],'ro')
plt.title("Training Result")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(testresult,y_data[ptraining:,:],'ro')
plt.title("Test Result")
plt.show()

In [ ]:
np.corrcoef(map(lambda x:x[0],trainingresult[0:ptraining].tolist()),
            map(lambda x:x[0],y_data[0:ptraining,:][0:ptraining].tolist()))[0][1]

In [ ]:
np.corrcoef(map(lambda x:x[0],testresult[0:240].tolist()),
         map(lambda x:x[0],y_data[ptraining:,:].tolist()))[0][1]

In [ ]:
map(lambda m:np.corrcoef(map(lambda x:x[0],trainingresult[m:ptraining:12].tolist()),
                         map(lambda x:x[0],y_data[0:ptraining,:][m:ptraining:12].tolist()))[0][1],
    range(12))

In [ ]:
map(lambda m:np.corrcoef(map(lambda x:x[0],testresult[m:240:12].tolist()),
                         map(lambda x:x[0],y_data[ptraining:,:][m:ptraining:12].tolist()))[0][1],
    range(12))